### 라이브러리

In [11]:
import pandas as pd
import os
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from gensim.summarization.summarizer import summarize
from newspaper import Article

### 데이터 전처리

In [ ]:
# # 파일에 있는 모든 csv 파일 불러오기

# # 불러온 파일이 있는 경로
# path = './부산_장애인_교통.csv'
# file_list = os.listdir(path)
# file_list_py = [file for file in file_list if file.endswith('.csv')] # 파일명이 끝이 .csv 인 경우

In [ ]:
# # csv 파일들을 DataFrame으로 불러와서 concat

# news = pd.DataFrame()
# for i in file_list_py:
#     data = pd.read_csv(path+'/'+i, encoding='utf-8')
#     news = pd.concat([news, data])
    
#     news = news.reset_index(drop=True)
# news

In [18]:
news = pd.read_csv('./data/저상버스_기사내용.csv', encoding='utf-8')
news

,Unnamed: 0.1,Unnamed: 0,기사제목,기사내용
0,0,0,"세계 장애인의 날 맞아 ""장애인권리 보장"" 집중투쟁",(서울=연합뉴스) 황철환 윤보람 기자 = 유엔이 정한 '제22회 세계 장애인...
1,1,1,‘장애인 고속버스’ 예산 16억이면 되는데…시범사업 물 건너가,"[한겨레] 정부, 리프트 설치 비용 내년 예산에 반영 안해장애인단체, 고속버스터미..."
2,2,2,출퇴근 대란 해결사 ‘2층버스’ 직접 타보니,[경기] 수원역에서 사당역까지 운행하는 7770번 좌석버스가 수원역 정류장에 당도...
3,3,3,"권철현 후보, 여성부시장제·여성일자리 1만개' 등 공약",【부산=뉴시스】허상천 기자 = 새누리당 권철현 부산시장 예비후보는 10일 “여성 부...
4,4,4,부산시청서 지하철서…장애인 차별 철폐 외친다,[한겨레] 34개 단체 자립권·이동권 투쟁시에 교육·복지·의료지원 요구부산민중연대...
...,...,...,...,...
150,150,150,"부산시, 대중교통 수송분담률 60%까지 끌어올린다",부산시 ‘대중교통 비전 2030’ 2008년 후 계속 40%대 그쳐 하단~녹산선·씨...
151,151,151,2026년까지 전국 저상 시내버스 도입률 62%로 높아진다,"국토부, 4차 교통약자 이동편의 증진계획(2022~2026년) 고시내년 1월 시내·..."
152,152,152,[전국 주요 신문 톱뉴스](28일 조간),▲ 경향신문 = 캄보디아 노동자 594편의 '절망기' ▲ 국민일보 = 한·미 ...
153,153,153,[시론] ‘약자복지’ 사각지대,빈곤층·장애인 잇따라 극단 선택경제적 자립 지원·일자리 창출 실패기초생활수급자 정책...


In [19]:
# news = news.drop(columns=['Unnamed: 0','날짜','기사제목'], axis=0)
# news = news.drop(columns=['Unnamed: 0','날짜'], axis=0)
news = news.drop(columns=['Unnamed: 0','기사제목', 'Unnamed: 0.1'], axis=0)
news["기사내용"] = news["기사내용"].str.replace("\t", "").str.replace(".", "\n")
news

C:\Users\admin\AppData\Local\Temp\ipykernel_12368\1549576533.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  news["기사내용"] = news["기사내용"].str.replace("\t", "").str.replace(".", "\n")


,기사내용
0,(서울=연합뉴스) 황철환 윤보람 기자 = 유엔이 정한 '제22회 세계 장애인...
1,"[한겨레] 정부, 리프트 설치 비용 내년 예산에 반영 안해장애인단체, 고속버스터미..."
2,[경기] 수원역에서 사당역까지 운행하는 7770번 좌석버스가 수원역 정류장에 당도...
3,【부산=뉴시스】허상천 기자 = 새누리당 권철현 부산시장 예비후보는 10일 “여성 부...
4,[한겨레] 34개 단체 자립권·이동권 투쟁시에 교육·복지·의료지원 요구부산민중연대...
...,...
150,부산시 ‘대중교통 비전 2030’ 2008년 후 계속 40%대 그쳐 하단~녹산선·씨...
151,"국토부, 4차 교통약자 이동편의 증진계획(2022~2026년) 고시내년 1월 시내·..."
152,▲ 경향신문 = 캄보디아 노동자 594편의 '절망기' ▲ 국민일보 = 한·미 ...
153,빈곤층·장애인 잇따라 극단 선택경제적 자립 지원·일자리 창출 실패기초생활수급자 정책...


In [20]:
# 요약 글 데이터프레임으로 만들어주기
for i in range(0, len(news)):
    try:
        text = news['기사내용'][i]
        # 문자열로 만들어주기
        text = (''.join(summarize(text, ratio=0.05).split('\n')))
        # print(text_list)
    except Exception as e:
        print(i,",", end='')
        continue

92 ,

In [21]:
news = news.drop([92])
news = news.reset_index(drop=True)
news

,기사내용
0,(서울=연합뉴스) 황철환 윤보람 기자 = 유엔이 정한 '제22회 세계 장애인...
1,"[한겨레] 정부, 리프트 설치 비용 내년 예산에 반영 안해장애인단체, 고속버스터미..."
2,[경기] 수원역에서 사당역까지 운행하는 7770번 좌석버스가 수원역 정류장에 당도...
3,【부산=뉴시스】허상천 기자 = 새누리당 권철현 부산시장 예비후보는 10일 “여성 부...
4,[한겨레] 34개 단체 자립권·이동권 투쟁시에 교육·복지·의료지원 요구부산민중연대...
...,...
149,부산시 ‘대중교통 비전 2030’ 2008년 후 계속 40%대 그쳐 하단~녹산선·씨...
150,"국토부, 4차 교통약자 이동편의 증진계획(2022~2026년) 고시내년 1월 시내·..."
151,▲ 경향신문 = 캄보디아 노동자 594편의 '절망기' ▲ 국민일보 = 한·미 ...
152,빈곤층·장애인 잇따라 극단 선택경제적 자립 지원·일자리 창출 실패기초생활수급자 정책...


In [22]:
# 리스트로 담기
sum_list = []

for i in range(0, len(news)):
    text = news['기사내용'][i]
    text_list = (''.join(summarize(text, ratio=0.8).split('\n')))
    
    sum_list.append(text_list)

In [23]:
total = pd.DataFrame(sum_list, columns=['기사요약'])
total

,기사요약
0,(서울=연합뉴스) 황철환 윤보람 기자 = 유엔이 정한 '제22회 세계 장애인의 날'...
1,"[한겨레] 정부, 리프트 설치 비용 내년 예산에 반영 안해장애인단체, 고속버스터미널..."
2,[경기] 수원역에서 사당역까지 운행하는 7770번 좌석버스가 수원역 정류장에 당도...
3,【부산=뉴시스】허상천 기자 = 새누리당 권철현 부산시장 예비후보는 10일 “여성 부...
4,[한겨레] 34개 단체 자립권·이동권 투쟁시에 교육·복지·의료지원 요구부산민중연대 ...
...,...
149,부산시 ‘대중교통 비전 2030’ 2008년 후 계속 40%대 그쳐 하단~녹산선·씨...
150,"국토부, 4차 교통약자 이동편의 증진계획(2022~2026년) 고시내년 1월 시내·..."
151,
152,빈곤층·장애인 잇따라 극단 선택경제적 자립 지원·일자리 창출 실패기초생활수급자 정책...


In [24]:
total.to_csv('./res/저상버스_기사내용.csv')